
### **Introduction**:

This notebook aims to train a model to detect road surface damage on Singapore roads. 

Research on damage detection of road surfaces using image processing techniques has been actively conducted on a range of datasets:
- CRACK500
- GAPs384
- CFD
- AEL
- cracktree200

The main motivation of this project is to simplify and reduce the manual effort of identifying defects by road inspectors. To ensure the ease of implementation, a smartphone based image recording and detection is preferred. As there are no readily available road damage dataset available for Singapore, the dataset collected by sekilab will be used. 

The dataset comprises 9,053 road damage images captured with a smartphone installed on a car, with 15,435 instances of road surface damage. The data was acquired across 7 municipalities in Japan. While road and weather conditions are different between Japan and Singapore, it presents a good start in automating road condition management in Singapore.

### **Road Damage Detection Using Tensorflow Object Detection API**

## Choosing a pre training model
The model used for this project is `ssd_mobilenet_v2_coco`.
Check other models from [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models).

Because the interestes of this project is to interfere on real time video, i am chosing a model that has a high inference speed `(ms)` with relativly high `mAP` on COCO

In [0]:


# Some models to train on
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_coco_2018_03_29',
        'pipeline_file': 'ssd_mobilenet_v2_coco.config',
    },
    'faster_rcnn_inception_v2': {
        'model_name': 'faster_rcnn_inception_v2_coco_2018_01_28',
        'pipeline_file': 'faster_rcnn_inception_v2_pets.config',
    },
    'rfcn_resnet101': {
        'model_name': 'rfcn_resnet101_coco_2018_01_28',
        'pipeline_file': 'rfcn_resnet101_pets.config',
    }
}

# Select a model in `MODELS_CONFIG`.
# I chose ssd_mobilenet_v2 for this project, you could choose any
selected_model = 'ssd_mobilenet_v2'


## Installing Required Packages 

In [2]:
!apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -qq Cython contextlib2 pillow lxml matplotlib

!pip install -qq pycocotools

Selecting previously unselected package python-bs4.
(Reading database ... 144568 files and directories currently installed.)
Preparing to unpack .../0-python-bs4_4.6.0-1_all.deb ...
Unpacking python-bs4 (4.6.0-1) ...
Selecting previously unselected package python-pkg-resources.
Preparing to unpack .../1-python-pkg-resources_39.0.1-2_all.deb ...
Unpacking python-pkg-resources (39.0.1-2) ...
Selecting previously unselected package python-chardet.
Preparing to unpack .../2-python-chardet_3.0.4-1_all.deb ...
Unpacking python-chardet (3.0.4-1) ...
Selecting previously unselected package python-six.
Preparing to unpack .../3-python-six_1.11.0-2_all.deb ...
Unpacking python-six (1.11.0-2) ...
Selecting previously unselected package python-webencodings.
Preparing to unpack .../4-python-webencodings_0.5-2_all.deb ...
Unpacking python-webencodings (0.5-2) ...
Selecting previously unselected package python-html5lib.
Preparing to unpack .../5-python-html5lib_0.999999999-1_all.deb ...
Unpacking pyt

In [3]:
!pip install numpy==1.17.4

     |████████████████████████████████| 20.0MB 69.9MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.18.2
    Uninstalling numpy-1.18.2:
      Successfully uninstalled numpy-1.18.2


## General imports
Other Imports will be done after downloading some packages later.

In [4]:
from __future__ import division, print_function, absolute_import

import pandas as pd
# numpy 1.18.2 doesn't work well, downgrade to 1.17.4. This works with tensorflow 1.15.2
!pip install numpy==1.17.4
import numpy as np
print(np.__version__)
import csv
import re
import cv2 
import os
import glob
import xml.etree.ElementTree as ET

import io

from PIL import Image
from collections import namedtuple, OrderedDict

import shutil
import urllib.request
import tarfile

from google.colab import files

1.18.2


In [5]:
#we need tenorflow v 1.15.0, object detection API is removed from tf v 2.0+
%tensorflow_version 1.x
import tensorflow.compat.v1 as tf
print(tf.__version__)



TensorFlow 1.x selected.
1.15.2


## Downloading and Orgniazing Images and Annotations
1. Downloading the images and annotations from the [source](https://sci2s.ugr.es/weapons-detection)  and unziping them
2. Creating a directory `(data)` to save some data such as; images, annotation, csv, etc...
3. Creating two directories; for the training and testing labels (not the images)
4. Randomly splitting our labels into 80% training and 20% testing and moving the splits to their directories: `(train_labels)` & `(test_labels)` 

In [0]:
!rm -rf road_defect_detection

In [0]:
#creates a directory for the whole project
!mkdir road_defect_detection

In [8]:
cd road_defect_detection

/content/road_defect_detection


In [9]:
#Training images and annotations

#Source: https://github.com/sekilab/RoadDamageDetector


#download the dataset zip
!wget https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/RoadDamageDatasetV2.zip

#unzip the files (contains images and annotation)
!unzip -q RoadDamageDatasetV2.zip


--2020-04-15 03:23:44--  https://mycityreport.s3-ap-northeast-1.amazonaws.com/02_RoadDamageDataset/RoadDamageDatasetV2.zip
Resolving mycityreport.s3-ap-northeast-1.amazonaws.com (mycityreport.s3-ap-northeast-1.amazonaws.com)... 52.219.16.79
Connecting to mycityreport.s3-ap-northeast-1.amazonaws.com (mycityreport.s3-ap-northeast-1.amazonaws.com)|52.219.16.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2280563788 (2.1G) [application/zip]
Saving to: ‘RoadDamageDatasetV2.zip’

RoadDamageDatasetV2 100%[===================>]   2.12G  13.5MB/s    in 2m 47s  

2020-04-15 03:26:32 (13.0 MB/s) - ‘RoadDamageDatasetV2.zip’ saved [2280563788/2280563788]



In [0]:
# creating a directory to store the training and testing data
!mkdir data

# folders for the training and testing data.
!mkdir data/images data/train_labels data/test_labels


# combining the images and annotation in the training folder:
# moves the images to data folder
!mv images/* data/images

# moves the annotations to data folder
!mv annotations/xmls/* data/train_labels

In [0]:
# Deleting the zipped and unzipped folders 
!rm -rf RoadDamageDatasetV2.zip

In [12]:
pwd

'/content/road_defect_detection'

In [13]:
import os
dir = 'data/images'
list_dir = os.listdir(dir) # dir is your directory path
number_files = len(list_dir)
print (number_files)
# train test split
traine = int(number_files*0.8)
teste = int(number_files*0.2)
print(traine)
print(teste)


10937
8749
2187


In [0]:
# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first teste labels to the testing dir: `test_labels`
!ls data/train_labels/* | sort -R | head -2187 | xargs -I{} mv {} data/test_labels

In [15]:
# 80% "images"(xml) for training
ls -1 data/train_labels/ | wc -l

8470


In [16]:
# 20% "images"(xml) for testing
ls -1 data/test_labels/ | wc -l

2187


## Preprocessing Images and Labels
1. Converting the annotations from xml files to two csv files for each `train_labels/` and `train_labels/`.
2. Creating a pbtxt file that specifies the number of class (one class in this case)
3. Checking if the annotations for each object are placed within the range of the image width and height.

In [17]:
pwd

'/content/road_defect_detection'

In [18]:

#adjusted from: https://github.com/datitran/raccoon_dataset

#converts the annotations/labels into one csv file for each training and testing labels
#creats label_map.pbtxt file

%cd /content/road_defect_detection/data


# images extension
images_extension = 'jpg'

# takes the path of a directory that contains xml files and converts
#  them to one csv file.

# returns a csv file that contains: image name, width, height, class, xmin, ymin, xmax, ymax.
# note: if the xml file contains more than one box/label, it will create more than one row for the same image. each row contains the info for an individual box. 
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member.find('name').text)
      value = (root.find('filename').text,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member.find('name').text,
               int(member.find('bndbox')[0].text),
               int(member.find('bndbox')[1].text),
               int(member.find('bndbox')[2].text),
               int(member.find('bndbox')[3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name) 
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

# for both the train_labels and test_labels csv files, it runs the xml_to_csv() above.
for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(image_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

# Creating the `label_map.pbtxt` file
label_map_path = os.path.join("label_map.pbtxt")

pbtxt_content = ""
classes_name = ['linear crack', 'alligator crack', 'rutting/bump/pothole', 
                'white line blur', 'cross walk blur', 'defect']
#creats a pbtxt file the has the class names.
for i, class_name in enumerate(classes):
    # display_name is optional.
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    name: '{1}'\n    id: {0}\n }}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)


/content/road_defect_detection/data
Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.


In [19]:
#checking the pbtxt file
!cat label_map.pbtxt

item {
    name: 'D00'
    id: 1
 }

item {
    name: 'D20'
    id: 2
 }

item {
    name: 'D40'
    id: 3
 }

item {
    name: 'D43'
    id: 4
 }

item {
    name: 'D44'
    id: 5
 }

item {
    name: 'D50'
    id: 6
 }

In [20]:
# they are there!
ls -l

total 2352
drwxr-xr-x 2 root root 561152 Apr 15 03:27 images/
-rw-r--r-- 1 root root    220 Apr 15 03:27 label_map.pbtxt
drwxr-xr-x 2 root root 126976 Apr 15 03:27 test_labels/
-rw-r--r-- 1 root root 237910 Apr 15 03:27 test_labels.csv
drwxr-xr-x 2 root root 528384 Apr 15 03:27 train_labels/
-rw-r--r-- 1 root root 938830 Apr 15 03:27 train_labels.csv


In [21]:
#checks if the images box position is placed within the image.

#note: while this doesn't checks if the boxes/annotatoins are correctly
# placed around the object, Tensorflow will through an error if this occured.
%cd /content/road_defect_detection/data
# path to images
images_path = 'images'

#loops over both train_labels and test_labels csv files to do the check
# returns the image name where an error is found 
# return the incorrect attributes; xmin, ymin, xmax, ymax.
for CSV_FILE in ['train_labels.csv', 'test_labels.csv']:
  with open(CSV_FILE, 'r') as fid:  
      print('[*] Checking file:', CSV_FILE) 
      file = csv.reader(fid, delimiter=',')
      first = True 
      cnt = 0
      error_cnt = 0
      error = False
      for row in file:
          if error == True:
              error_cnt += 1
              error = False         
          if first == True:
              first = False
              continue     
          cnt += 1      
          name, width, height, xmin, ymin, xmax, ymax = row[0], int(row[1]), int(row[2]), int(row[4]), int(row[5]), int(row[6]), int(row[7])     
          path = os.path.join(images_path, name)
          img = cv2.imread(path)         
          if type(img) == type(None):
              error = True
              print('Could not read image', img)
              continue     
          org_height, org_width = img.shape[:2]     
          if org_width != width:
              error = True
              print('Width mismatch for image: ', name, width, '!=', org_width)     
          if org_height != height:
              error = True
              print('Height mismatch for image: ', name, height, '!=', org_height) 
          if xmin > org_width:
              error = True
              print('XMIN > org_width for file', name)  
          if xmax > org_width:
              error = True
              print('XMAX > org_width for file', name)
          if ymin > org_height:
              error = True
              print('YMIN > org_height for file', name)
          if ymax > org_height:
              error = True
              print('YMAX > org_height for file', name)
          if error == True:
              print('Error for file: %s' % name)
              print()
      print()
      print('Checked %d files and realized %d errors' % (cnt, error_cnt))
      print("-----")

/content/road_defect_detection/data
[*] Checking file: train_labels.csv

Checked 17314 files and realized 0 errors
-----
[*] Checking file: test_labels.csv

Checked 4391 files and realized 0 errors
-----


In [0]:
#if any images have incorrect box position, we could just remove it 
#removing the image 
#rm images/'xxxx.jpg'

In [23]:
#removing the entry for it in the csv for that image as well

#because we did a random split for the data, we dont know if it ended up being in training or testing
# we will remove the image from both.

#training
#reading the training csv
'''
df = pd.read_csv('/content/road_defect_detection/data/train_labels.csv')
# removing xxxx.jpg
df = df[df['filename'] != 'xxxx.jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/road_defect_detection/data/train_labels.csv')


#testing
#reading the testing csv
df = pd.read_csv('/content/road_defect_detection/data/test_labels.csv')
# removing xxxx.jpg
df = df[df['filename'] != 'xxxx.jpg']
#reseting the index
df.reset_index(drop=True, inplace=True)
#saving the df
df.to_csv('/content/road_defect_detection/data/test_labels.csv')

# Just for the memory
df = None
'''

"\ndf = pd.read_csv('/content/road_defect_detection/data/train_labels.csv')\n# removing xxxx.jpg\ndf = df[df['filename'] != 'xxxx.jpg']\n#reseting the index\ndf.reset_index(drop=True, inplace=True)\n#saving the df\ndf.to_csv('/content/road_defect_detection/data/train_labels.csv')\n\n\n#testing\n#reading the testing csv\ndf = pd.read_csv('/content/road_defect_detection/data/test_labels.csv')\n# removing xxxx.jpg\ndf = df[df['filename'] != 'xxxx.jpg']\n#reseting the index\ndf.reset_index(drop=True, inplace=True)\n#saving the df\ndf.to_csv('/content/road_defect_detection/data/test_labels.csv')\n\n# Just for the memory\ndf = None\n"

## Downloading and Preparing Tensorflow model
1. Cloning [Tensorflow models](https://github.com/tensorflow/models.git) from the offical git repo. The repo contains the object detection API we are interseted in. 
2. Compiling the protos and adding folders to the os environment.
3. Testing the model builder.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [25]:
# Downlaods Tensorflow
%cd /content/road_defect_detection/
!git clone --q https://github.com/tensorflow/models.git

/content/road_defect_detection


In [26]:
%cd /content/road_defect_detection/models/research
#compiling the proto buffers (not important to understand for this project but you can learn more about them here: https://developers.google.com/protocol-buffers/)
!protoc object_detection/protos/*.proto --python_out=.

# exports the PYTHONPATH environment variable with the reasearch and slim folders' paths
os.environ['PYTHONPATH'] += ':/content/road_defect_detection/models/research/:/content/road_defect_detection/models/research/slim/'

/content/road_defect_detection/models/research


In [27]:
# testing the model builder
!python3 object_detection/builders/model_builder_test.py

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Running tests under Python 3.6.9: /usr/bin/python3
[ RUN      ] ModelBuilderTest.test_create_experimental_model
[       OK ] ModelBuilderTest.test_create_experimental_model
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[       OK ] ModelBuilderTest.test_create_faster_rcnn_model_from_config_with_example_miner
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[       OK ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_with_matmul
[ RUN      ] ModelBuilderTest.test_create_faster_rcnn_models_from_config_faster_rcnn_wi

## Generating Tf record
- Generating two TFRecords files for the training and testing CSVs.
- Tensorflow accepts the data as tfrecords which is a binary file that run fast with low memory usage. Instead of loading the full data into memory, Tenorflow breaks the data into batches using these TFRecords automatically

In [28]:
#adjusted from: https://github.com/EdjeElectronics/TensorFlow-Object-Detection-API-Tutorial-Train-Multiple-Objects-Windows-10/blob/master/generate_tfrecord.py

# converts the csv files for training and testing data to two TFRecords files.
# places the output in the same directory as the input


from object_detection.utils import dataset_util
%cd /content/road_defect_detection/models/

DATA_BASE_PATH = '/content/road_defect_detection/data/'
image_dir = DATA_BASE_PATH +'images/'

def class_text_to_int(row_label):
    if row_label == 'D00':
        return 1
    elif row_label == 'D20':
        return 2
    elif row_label == 'D40':
        return 3
    elif row_label == 'D43':
        return 4
    elif row_label == 'D44':
        return 5
    elif row_label == 'D50':
        return 6
    else:
        None


def split(df, group):
		data = namedtuple('data', ['filename', 'object'])
		gb = df.groupby(group)
		return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

def create_tf_example(group, path):
		with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
				encoded_jpg = fid.read()
		encoded_jpg_io = io.BytesIO(encoded_jpg)
		image = Image.open(encoded_jpg_io)
		width, height = image.size

		filename = group.filename.encode('utf8')
		image_format = b'jpg'
		xmins = []
		xmaxs = []
		ymins = []
		ymaxs = []
		classes_text = []
		classes = []

		for index, row in group.object.iterrows():
				xmins.append(row['xmin'] / width)
				xmaxs.append(row['xmax'] / width)
				ymins.append(row['ymin'] / height)
				ymaxs.append(row['ymax'] / height)
				classes_text.append(row['class'].encode('utf8'))
				classes.append(class_text_to_int(row['class']))

		tf_example = tf.train.Example(features=tf.train.Features(feature={
				'image/height': dataset_util.int64_feature(height),
				'image/width': dataset_util.int64_feature(width),
				'image/filename': dataset_util.bytes_feature(filename),
				'image/source_id': dataset_util.bytes_feature(filename),
				'image/encoded': dataset_util.bytes_feature(encoded_jpg),
				'image/format': dataset_util.bytes_feature(image_format),
				'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
				'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
				'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
				'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
				'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
				'image/object/class/label': dataset_util.int64_list_feature(classes),
		}))
		return tf_example

for csv in ['train_labels', 'test_labels']:
  writer = tf.io.TFRecordWriter(DATA_BASE_PATH + csv + '.record')
  path = os.path.join(image_dir)
  examples = pd.read_csv(DATA_BASE_PATH + csv + '.csv')
  grouped = split(examples, 'filename')
  for group in grouped:
      tf_example = create_tf_example(group, path)
      writer.write(tf_example.SerializeToString())
    
  writer.close()
  output_path = os.path.join(os.getcwd(), DATA_BASE_PATH + csv + '.record')
  print('Successfully created the TFRecords: {}'.format(DATA_BASE_PATH +csv + '.record'))


/content/road_defect_detection/models
Successfully created the TFRecords: /content/road_defect_detection/data/train_labels.record
Successfully created the TFRecords: /content/road_defect_detection/data/test_labels.record


In [29]:
# TFRecords are created
ls -lX /content/road_defect_detection/data/

total 1986108
drwxr-xr-x 2 root root     561152 Apr 15 03:27 images/
drwxr-xr-x 2 root root     126976 Apr 15 03:27 test_labels/
drwxr-xr-x 2 root root     528384 Apr 15 03:27 train_labels/
-rw-r--r-- 1 root root     237910 Apr 15 03:27 test_labels.csv
-rw-r--r-- 1 root root     938830 Apr 15 03:27 train_labels.csv
-rw-r--r-- 1 root root        220 Apr 15 03:27 label_map.pbtxt
-rw-r--r-- 1 root root  420177375 Apr 15 03:34 test_labels.record
-rw-r--r-- 1 root root 1611179914 Apr 15 03:34 train_labels.record


## Downloading the Base Model
1. Based on the model selecting at the top of this notebook, downloading the model selected and extracting its content.
2. Creating a dir to save the model while training.

In [30]:
%cd /content/road_defect_detection/models/research

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

#selecting the model
MODEL_FILE = MODEL + '.tar.gz'

#creating the downlaod link for the model selected
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'

#the distination folder where the model will be saved
fine_tune_dir = '/content/road_defect_detection/models/research/pretrained_model'

#checks if the model has already been downloaded
if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

#unzipping the file and extracting its content
tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

# creating an output file to save the model while training
os.remove(MODEL_FILE)
if (os.path.exists(fine_tune_dir)):
    shutil.rmtree(fine_tune_dir)
os.rename(MODEL, fine_tune_dir)

/content/road_defect_detection/models/research


In [31]:
#checking the content of the pretrained model.
# this is the directory of the "fine_tune_checkpoint" that is used in the config file.
!echo {fine_tune_dir}
!ls -alh {fine_tune_dir}

/content/road_defect_detection/models/research/pretrained_model
total 135M
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 .
drwxr-xr-x 63 root   root  4.0K Apr 15 03:34 ..
-rw-r--r--  1 345018 89939   77 Mar 30  2018 checkpoint
-rw-r--r--  1 345018 89939  67M Mar 30  2018 frozen_inference_graph.pb
-rw-r--r--  1 345018 89939  65M Mar 30  2018 model.ckpt.data-00000-of-00001
-rw-r--r--  1 345018 89939  15K Mar 30  2018 model.ckpt.index
-rw-r--r--  1 345018 89939 3.4M Mar 30  2018 model.ckpt.meta
-rw-r--r--  1 345018 89939 4.2K Mar 30  2018 pipeline.config
drwxr-xr-x  3 345018 89939 4.0K Mar 30  2018 saved_model


## Configuring the Training Pipeline
1. Adding the path for the TFRecords files and pbtxt,batch_size,num_steps,num_classes to the configuration file.
2. Adding some Image augmentation.
3. Creating a directory to save the model at each checkpoint while training. 

In [32]:

#the path to the folder containing all the sample config files
CONFIG_BASE = "/content/road_defect_detection/models/research/object_detection/samples/configs/"

#path to the specified model's config file
model_pipline = os.path.join(CONFIG_BASE, pipeline_file)
model_pipline

'/content/road_defect_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config'

In [33]:
#check the sample config file that is provided by the tf model
!cat /content/road_defect_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config

# SSD with Mobilenet v2 configuration for MSCOCO Dataset.
# Users should configure the fine_tune_checkpoint field in the train config as
# well as the label_map_path and input_path fields in the train_input_reader and
# eval_input_reader. Search for "PATH_TO_BE_CONFIGURED" to find the fields that
# should be configured.

model {
  ssd {
    num_classes: 90
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect

In [34]:
#editing the configuration file to add the path for the TFRecords files, pbtxt,batch_size,num_steps,num_classes.
# any image augmentation, hyperparemeter tunning (drop out, batch normalization... etc) would be editted here

%%writefile {model_pipline}
model {
  ssd {
    num_classes: 6 # number of classes to be detected
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
        width_scale: 5.0
      }
    }
    matcher {
      argmax_matcher {
        matched_threshold: 0.5
        unmatched_threshold: 0.5
        ignore_thresholds: false
        negatives_lower_than_unmatched: true
        force_match_for_each_row: true
      }
    }
    similarity_calculator {
      iou_similarity {
      }
    }
    anchor_generator {
      ssd_anchor_generator {
        num_layers: 6
        min_scale: 0.2
        max_scale: 0.95
        aspect_ratios: 1.0
        aspect_ratios: 2.0
        aspect_ratios: 0.5
        aspect_ratios: 3.0
        aspect_ratios: 0.3333
      }
    }
    # all images will be resized to the below W x H.
    image_resizer { 
      fixed_shape_resizer {
        height: 300
        width: 300
      }
    }
    box_predictor {
      convolutional_box_predictor {
        min_depth: 0
        max_depth: 0
        num_layers_before_predictor: 0
        #use_dropout: false
        use_dropout: true # to counter over fitting. you can also try tweaking its probability below
        dropout_keep_probability: 0.8
        kernel_size: 1
        box_code_size: 4
        apply_sigmoid_to_scores: false
        conv_hyperparams {
          activation: RELU_6,
          regularizer {
            l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
          }
          initializer {
            truncated_normal_initializer {
              stddev: 0.03
              mean: 0.0
            }
          }
          batch_norm {
            train: true,
            scale: true,
            center: true,
            decay: 0.9997,
            epsilon: 0.001,
          }
        }
      }
    }
    feature_extractor {
      type: 'ssd_mobilenet_v2'
      min_depth: 16
      depth_multiplier: 1.0
      conv_hyperparams {
        activation: RELU_6,
        regularizer {
          l2_regularizer {
            # weight: 0.00004
            weight: 0.001 # higher regularizition to counter overfitting
          }
        }
        initializer {
          truncated_normal_initializer {
            stddev: 0.03
            mean: 0.0
          }
        }
        batch_norm {
          train: true,
          scale: true,
          center: true,
          decay: 0.9997,
          epsilon: 0.001,
        }
      }
    }
    loss {
      classification_loss {
        weighted_sigmoid {
        }
      }
      localization_loss {
        weighted_smooth_l1 {
        }
      }
      hard_example_miner {
        num_hard_examples: 3000 
        iou_threshold: 0.95
        loss_type: CLASSIFICATION
        max_negatives_per_positive: 3
        min_negatives_per_image: 3
      }
      classification_weight: 1.0
      localization_weight: 1.0
    }
    normalize_loss_by_num_matches: true
    post_processing {
      batch_non_max_suppression {
        score_threshold: 1e-8
        iou_threshold: 0.6
        
        #adjust this to the max number of objects per class. 
        # ex, in my case, i have one road defect in most of the images.
        # . there are some images with more than one
        max_detections_per_class: 5
        # max number of detections among all classes. I have 6 class so
        # doesn't actually make sense to have so many defects in one frame
        # due to lack of familiarity with images, just setting to 30
        max_total_detections: 5
      }
      score_converter: SIGMOID
    }
  }
}

train_config: {
  batch_size: 16 # training batch size
  optimizer {
    rms_prop_optimizer: {
      learning_rate: {
        exponential_decay_learning_rate {
          initial_learning_rate: 0.003
          decay_steps: 800720
          decay_factor: 0.95
        }
      }
      momentum_optimizer_value: 0.9
      decay: 0.9
      epsilon: 1.0
    }
  }

  #the path to the pretrained model. 
  fine_tune_checkpoint: "/content/road_defect_detection/models/research/pretrained_model/model.ckpt"
  fine_tune_checkpoint_type:  "detection"
  # Note: The below line limits the training process to 200K steps, which we
  # empirically found to be sufficient enough to train the pets dataset. This
  # effectively bypasses the learning rate schedule (the learning rate will
  # never decay). Remove the below line to train indefinitely.
  num_steps: 200000 
  

  #data augmentaion is done here, you can remove or add more.
  # They will help the model generalize but the training time will increase greatly by using more data augmentation.
  # Check this link to add more image augmentation: https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto
  
  data_augmentation_options {
    random_horizontal_flip {
    }
  }
  data_augmentation_options {
    random_adjust_contrast {
    }
  }
  data_augmentation_options {
    ssd_random_crop {
    }
  }
}

train_input_reader: {
  tf_record_input_reader {
    #path to the training TFRecord
    input_path: "/content/road_defect_detection/data/train_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/road_defect_detection/data/label_map.pbtxt"
}

eval_config: {
  # the number of images in your "testing" data (20% of data, recall teste :) )
  num_examples: 2187
  # the number of images to disply in Tensorboard while training
  num_visualizations: 20

  # Note: The below line limits the evaluation process to 10 evaluations.
  # Remove the below line to evaluate indefinitely.
  #max_evals: 10
}

eval_input_reader: {
  tf_record_input_reader {
      
    #path to the testing TFRecord
    input_path: "/content/road_defect_detection/data/test_labels.record"
  }
  #path to the label map 
  label_map_path: "/content/road_defect_detection/data/label_map.pbtxt"
  shuffle: false
  num_readers: 1
}

Overwriting /content/road_defect_detection/models/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config


In [0]:
# where the model will be saved at each checkpoint while training 
model_dir = 'training/'

# Optionally: remove content in output model directory to fresh start.
!rm -rf {model_dir}
os.makedirs(model_dir, exist_ok=True)

In [36]:
pwd

'/content/road_defect_detection/models/research'

## Tensorboard
1. Downloading and unzipping Tensorboard
2. creating a link to visualize multiple graph while training.


notes: 
  1. Tensorboard will not log any files until the training starts. 
  2. a max of 20 connection per minute is allowed when using ngrok, you will not be able to access tensorboard while the model is logging.

In [37]:

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip -o ngrok-stable-linux-amd64.zip

--2020-04-15 03:34:34--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.197.27.35, 34.196.12.177, 3.229.196.117, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.197.27.35|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  14.0MB/s    in 0.9s    

2020-04-15 03:34:35 (14.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [38]:
# specify wher log files are stored and configure link to view tensorboard
#the logs that are created while training 
LOG_DIR = "/content/road_defect_detection/models/research/training/"
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)
get_ipython().system_raw('./ngrok http 6006 &')
#The link to tensorboard.
#works after the training starts.
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://a14a26fc.ngrok.io


## Training

Finally training the model!


In [0]:

!python3 /content/road_defect_detection/models/research/object_detection/model_main.py \
    --pipeline_config_path={model_pipline}\
    --model_dir={model_dir} \
    --alsologtostderr \

Streaming output truncated to the last 5000 lines.
I0415 06:27:02.723882 140251547891584 estimator.py:2109] Saving 'checkpoint_path' summary for global step 20977: training/model.ckpt-20977
INFO:tensorflow:global_step/sec: 0.743089
I0415 06:27:13.161934 140251547891584 basic_session_run_hooks.py:692] global_step/sec: 0.743089
INFO:tensorflow:loss = 5.5043235, step = 21000 (134.573 sec)
I0415 06:27:13.163000 140251547891584 basic_session_run_hooks.py:260] loss = 5.5043235, step = 21000 (134.573 sec)
INFO:tensorflow:global_step/sec: 2.33402
I0415 06:27:56.006461 140251547891584 basic_session_run_hooks.py:692] global_step/sec: 2.33402
INFO:tensorflow:loss = 5.107008, step = 21100 (42.844 sec)
I0415 06:27:56.007367 140251547891584 basic_session_run_hooks.py:260] loss = 5.107008, step = 21100 (42.844 sec)
INFO:tensorflow:global_step/sec: 2.33457
I0415 06:28:38.840908 140251547891584 basic_session_run_hooks.py:692] global_step/sec: 2.33457
INFO:tensorflow:loss = 5.9651003, step = 21200 (42.8

## Exporting The Trained model



In [0]:


#the location where the exported model will be saved in.
output_directory = '/content/road_defect_detection/models/research/fine_tuned_model'

# goes through the model is the training/ dir and gets the last one.
# you could choose a specfic one instead of the last
lst = os.listdir(model_dir)
lst = [l for l in lst if 'model.ckpt-' in l and '.meta' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()].replace('.meta', '')
last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)

#exports the model specifed and inference graph
!python /content/road_defect_detection/models/research/object_detection/export_inference_graph.py \
    --input_type=image_tensor \
    --pipeline_config_path={model_pipline} \
    --output_directory={output_directory} \
    --trained_checkpoint_prefix={last_model_path}

In [0]:
#downloads the frozen model that is needed for inference
files.download(output_directory + '/frozen_inference_graph.pb')

In [0]:
#downlaod the label map
files.download(DATA_BASE_PATH + '/label_map.pbtxt')

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%tensorboard --logdir logs